In [3]:
import requests
import time, csv, math

Słowem wstępu - zgodnie z regulaminem i warunkami korzystania z usługi, niniejszy raport został opracowany przy użyciu Klucza API udostępnionego przez The Movie Database (TMDB). Klucz API jest niezbędny do uzyskania dostępu do danych filmowych i analizy przeprowadzonej w ramach tego projektu.

Klucz API został otrzymany poprzez proces rejestracji i wnioskowania na stronie TMDB, zgodnie z wytycznymi udostępnionymi przez dostawcę usługi. Wniosek został złożony i zatwierdzony, a Klucz API został przypisany do tego projektu, umożliwiając dostęp do niezbędnych danych filmowych.

Wszelkie analizy, wyniki i prezentowane dane będą w tym raporcie oparte na danych udostępnionych przez TMDB przy użyciu Klucza API i zgodnie z dostępem, który mi został przyznany.

Wykorzystam poniższy skrypt, który napisałem, do uzyskania najpierw wszystkich numerów ID wybranej przeze mnie liczby filmów. Do mojego projektu, myślę, że wystarczy mi ich 10000, zatem właśnie tyle ich pobiorę.

*W parametrach jest pole **"api_key"**, którego wartość zakryłem, bez niego **nie uda się** pobrać danych filmowych.*

In [4]:
# Ustawienie parametrów wyszukiwania
base_url = "https://api.themoviedb.org/3"
endpoint = "/discover/movie"
params = {
    "api_key": "MY_API_KEY",
    "sort_by": "popularity.desc",
    "page": 1,
}

# Pobranie danych dla n filmów
n = 10000
total_pages = math.ceil(n / 20)
print(
    total_pages
)  # Zgodnie z dokumentacją TMDB na jednej stronie znajduje się 20 filmów

movies_ids = []
errors_in_a_row = 0
for i in range(1, total_pages + 1):
    params["page"] = i
    print(f"Retrieving page {i}/{total_pages}")
    url = f"{base_url}{endpoint}"
    response = requests.get(url, params=params)
    if response.status_code == 200:
        errors_in_a_row = 0
        data = response.json()
        if data["results"]:
            for movie in data["results"]:
                movies_ids.append(movie["id"])
                if len(movies_ids) >= n:
                    break
        if len(movies_ids) >= n:
            break

    elif response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After"))
        print(f"Query limit exceeded. Wait for {retry_after} seconds...")
        time.sleep(retry_after)
        i -= 1
    else:
        print(
            f"An error was encountered while fetching data from page {i}. Status code = {response.status_code}"
        )
        errors_in_a_row += 1
        if errors_in_a_row > 30:
            break

    if i % 10 == 0:
        wait_time = 2
        print(
            f"Waiting for {wait_time} seconds... There are currently {len(movies_ids)} movies IDs saved."
        )
        time.sleep(wait_time)

1000
Retrieving page 1/1000
Retrieving page 2/1000
Retrieving page 3/1000
Retrieving page 4/1000
Retrieving page 5/1000
Retrieving page 6/1000
Retrieving page 7/1000
Retrieving page 8/1000
Retrieving page 9/1000
Retrieving page 10/1000
Waiting for 2 seconds... There are currently 200 movies IDs saved.
Retrieving page 11/1000
Retrieving page 12/1000
Retrieving page 13/1000
Retrieving page 14/1000
Retrieving page 15/1000
Retrieving page 16/1000
Retrieving page 17/1000
Retrieving page 18/1000
Retrieving page 19/1000
Retrieving page 20/1000
Waiting for 2 seconds... There are currently 400 movies IDs saved.
Retrieving page 21/1000
Retrieving page 22/1000
Retrieving page 23/1000
Retrieving page 24/1000
Retrieving page 25/1000
Retrieving page 26/1000
Retrieving page 27/1000
Retrieving page 28/1000
Retrieving page 29/1000
Retrieving page 30/1000
Waiting for 2 seconds... There are currently 600 movies IDs saved.
Retrieving page 31/1000
Retrieving page 32/1000
Retrieving page 33/1000
Retrieving 

Otrzymaliśmy w ten sposób 10000 numerów ID filmów. Zajrzyjmy jak one wyglądają.

In [5]:
print(movies_ids)

[603692, 502356, 569094, 385687, 890771, 447277, 667538, 879444, 713704, 640146, 1073140, 76600, 840326, 447365, 882569, 1018494, 1107872, 536437, 324857, 1098110, 594767, 799379, 758323, 1077321, 605886, 552688, 1115710, 893712, 1074178, 677179, 1070777, 620705, 315162, 943930, 934433, 1064517, 842675, 868759, 1105803, 1118203, 1094319, 493529, 420808, 634649, 916224, 981324, 298618, 505642, 995012, 8869, 948713, 436270, 977866, 1094403, 638974, 60898, 649609, 1010581, 1102776, 702621, 816904, 804150, 1020696, 736790, 977223, 325358, 697843, 10144, 507250, 1121116, 970284, 982271, 1016121, 1119173, 700391, 536554, 646389, 1126852, 976912, 997776, 723347, 758336, 983105, 911916, 920125, 962682, 384018, 1037644, 91314, 13962, 980078, 876969, 1123342, 616037, 19995, 1033219, 675353, 876792, 785084, 1121316, 727340, 985939, 507086, 299536, 785759, 946310, 961718, 361743, 453395, 283995, 1129692, 943788, 1014501, 869626, 631842, 758009, 663712, 724495, 1003579, 1111140, 455476, 842945, 673

Dzięki posiadaniu dokładnego numeru ID każdego filmu, jesteśmy w stanie zebrać jego wszystkie szczegółowe informacje przy jednorazowym zapytaniu. Pobierzmy je więc.

In [6]:
movies_data = []

movie_endpoint = "/movie/{movie_id}"
movie_params = {
    "api_key": "MY_API_KEY",
    "append_to_response": "reviews,credits",
}

for i in range(len(movies_ids)):
    movie_id = movies_ids[i]
    movie_url = f"{base_url}{movie_endpoint}".format(movie_id=movie_id)
    movie_response = requests.get(movie_url, params=movie_params)
    if movie_response.status_code == 200:
        movie_data = movie_response.json()
        movies_data.append(movie_data)
        if (i + 1) % 10 == 0:
            print(f"Downloaded already {i+1} movies details")
    elif response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After"))
        print(f"Query limit exceeded. Waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        i -= 1
    else:
        print(
            f"An error was encountered while fetching data for movie with ID {movie_id}). Status code = {response.status_code}"
        )
        errors_in_a_row += 1
        if errors_in_a_row > 30:
            break
    if (i + 1) % 500 == 0:
        wait_time = 2
        print(
            f"Waiting for {wait_time} seconds... There are currently {len(movies_data)} additional data saved."
        )
        time.sleep(wait_time)

print(f"Saved details of {len(movies_data)} movies")

Downloaded already 10 movies details
Downloaded already 20 movies details
Downloaded already 30 movies details
Downloaded already 40 movies details
Downloaded already 50 movies details
Downloaded already 60 movies details
Downloaded already 70 movies details
Downloaded already 80 movies details
Downloaded already 90 movies details
Downloaded already 100 movies details
Downloaded already 110 movies details
Downloaded already 120 movies details
Downloaded already 130 movies details
Downloaded already 140 movies details
Downloaded already 150 movies details
Downloaded already 160 movies details
Downloaded already 170 movies details
Downloaded already 180 movies details
Downloaded already 190 movies details
Downloaded already 200 movies details
Downloaded already 210 movies details
Downloaded already 220 movies details
Downloaded already 230 movies details
Downloaded already 240 movies details
Downloaded already 250 movies details
Downloaded already 260 movies details
Downloaded already 27

Sprawdźmy jak wyglądają pobrane dane:

In [7]:
print(movies_data[0])
print(movies_data[1])
print(movies_data[2])

{'adult': False, 'backdrop_path': '/h8gHn0OzBoaefsYseUByqsmEDMY.jpg', 'belongs_to_collection': {'id': 404609, 'name': 'John Wick Collection', 'poster_path': '/xUidyvYFsbbuExifLkslpcd8SMc.jpg', 'backdrop_path': '/fSwYa5q2xRkBoOOjueLpkLf3N1m.jpg'}, 'budget': 90000000, 'genres': [{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}, {'id': 80, 'name': 'Crime'}], 'homepage': 'https://johnwick.movie', 'id': 603692, 'imdb_id': 'tt10366206', 'original_language': 'en', 'original_title': 'John Wick: Chapter 4', 'overview': 'With the price on his head ever increasing, John Wick uncovers a path to defeating The High Table. But before he can earn his freedom, Wick must face off against a new enemy with powerful alliances across the globe and forces that turn old friends into foes.', 'popularity': 4071.868, 'poster_path': '/vZloFAK7NmvMGKE7VkF5UHaz0I.jpg', 'production_companies': [{'id': 3528, 'logo_path': '/cCzCClIzIh81Fa79hpW5nXoUsHK.png', 'name': 'Thunder Road', 'origin_country': 'US'}, 

Mamy już wszystkie potrzebne dane w nieco chaotycznej jeszcze formie, ale zajmiemy się tym podczas preprocessingu. Póki co zapiszmy nasze dane do pliku CSV.

In [11]:
# Zapisywanie danych do pliku CSV
with open("movies_data.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=movies_data[0].keys())
    writer.writeheader()
    writer.writerows(movies_data)

print(f"Saved {len(movies_data)} movies to movies.csv")

Saved 10000 movies to movies.csv
